In [2]:
import pandas as pd

In [3]:
df = pd.read_csv("../data/final/Ghana_Maize_Data.csv")
df.head()

,District,Year,Yield,Total_Rainfall,Rainfall_Variability,Avg_Temp,Max_Temp,Avg_Humidity,Avg_Sunlight,Avg_Soil_Moisture,Min_Soil_Moisture,Soil_Type,Pest_Risk,PFJ_Policy,Yield_Lag1
0,A.M.A,2011,1.53,670.85,3.813231,25.852745,28.24,87.471373,15.939739,0.827974,0.74,Coastal Savannah,0,0,1.00
1,A.M.A,2012,1.85,683.60,5.012967,25.591503,28.02,86.877582,16.625163,0.814248,0.71,Coastal Savannah,0,0,1.53
2,A.M.A,2013,1.80,481.38,3.373025,25.802222,28.21,85.133856,16.894248,0.761242,0.66,Coastal Savannah,0,0,1.85
3,A.M.A,2014,1.34,596.92,4.603216,26.129346,28.48,85.771699,16.374771,0.782484,0.70,Coastal Savannah,0,0,1.80
4,A.M.A,2015,1.30,516.07,5.134346,26.004052,28.37,86.899477,16.843137,0.793595,0.70,Coastal Savannah,0,0,1.34


In [5]:
df.tail()

,District,Year,Yield,Total_Rainfall,Rainfall_Variability,Avg_Temp,Max_Temp,Avg_Humidity,Avg_Sunlight,Avg_Soil_Moisture,Min_Soil_Moisture,Soil_Type,Pest_Risk,PFJ_Policy,Yield_Lag1
1759,Zabzugu/Tatale,2017,2.04,949.71,8.431942,27.290850,31.98,77.446797,17.766471,0.559412,0.35,Savanna Ochrosol,1,1,1.90
1760,Zabzugu/Tatale,2018,2.25,777.88,6.998354,26.903922,32.04,78.688562,18.094118,0.548954,0.33,Savanna Ochrosol,0,1,2.04
1761,Zabzugu/Tatale,2019,2.53,798.27,10.247853,27.314248,33.46,78.131569,18.188301,0.544967,0.34,Savanna Ochrosol,0,1,2.25
1762,Zabzugu/Tatale,2020,2.58,838.97,8.454623,26.931373,32.60,79.364837,18.549085,0.565359,0.39,Savanna Ochrosol,0,1,2.53
1763,Zabzugu/Tatale,2021,2.53,781.71,5.684462,27.340392,31.95,76.147124,18.882549,0.539216,0.36,Savanna Ochrosol,0,1,2.58


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1764 entries, 0 to 1763
Data columns (total 15 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   District              1764 non-null   object 
 1   Year                  1764 non-null   int64  
 2   Yield                 1764 non-null   float64
 3   Total_Rainfall        1764 non-null   float64
 4   Rainfall_Variability  1764 non-null   float64
 5   Avg_Temp              1764 non-null   float64
 6   Max_Temp              1764 non-null   float64
 7   Avg_Humidity          1764 non-null   float64
 8   Avg_Sunlight          1764 non-null   float64
 9   Avg_Soil_Moisture     1764 non-null   float64
 10  Min_Soil_Moisture     1764 non-null   float64
 11  Soil_Type             1764 non-null   object 
 12  Pest_Risk             1764 non-null   int64  
 13  PFJ_Policy            1764 non-null   int64  
 14  Yield_Lag1            1764 non-null   float64
dtypes: float64(10), int64

In [6]:
df.duplicated().sum()

np.int64(0)

In [7]:
df.isna().sum()

District                0
Year                    0
Yield                   0
Total_Rainfall          0
Rainfall_Variability    0
Avg_Temp                0
Max_Temp                0
Avg_Humidity            0
Avg_Sunlight            0
Avg_Soil_Moisture       0
Min_Soil_Moisture       0
Soil_Type               0
Pest_Risk               0
PFJ_Policy              0
Yield_Lag1              0
dtype: int64

In [8]:
df.columns

Index(['District', 'Year', 'Yield', 'Total_Rainfall', 'Rainfall_Variability',
       'Avg_Temp', 'Max_Temp', 'Avg_Humidity', 'Avg_Sunlight',
       'Avg_Soil_Moisture', 'Min_Soil_Moisture', 'Soil_Type', 'Pest_Risk',
       'PFJ_Policy', 'Yield_Lag1'],
      dtype='object')

In [17]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, ExtraTreesRegressor, StackingRegressor
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, mean_absolute_error
import joblib


df = pd.read_csv("../data/final/Ghana_Maize_Data.csv")

X = df[['Year', 'Rainfall',
       'Temperature', 'Humidity', 'Sunlight',
       'Soil_Moisture', 'Soil_Type', 'Pest_Risk',
       'PFJ_Policy', 'Yield_Lag1']]
y = df['Yield']

numeric_cols = [c for c in X.columns if c != "Soil_Type"]
categorical_cols = ["Soil_Type"]

preprocessor = ColumnTransformer([
    ('num', StandardScaler(), numeric_cols),
    ('cat', OneHotEncoder(handle_unknown="ignore"), categorical_cols)
])

# Strong base learners
estimators = [
    ('rf', RandomForestRegressor(
        n_estimators=300, random_state=299, min_samples_split=2
    )),
    ('gb', GradientBoostingRegressor(
        n_estimators=300, learning_rate=0.05, random_state=299
    )),
    ('et', ExtraTreesRegressor(
        n_estimators=300, random_state=299
    ))
]

# Meta-learner learns how to combine base models
stacked = StackingRegressor(
    estimators=estimators,
    final_estimator=LinearRegression(),
    passthrough=True
)

model_pipeline = Pipeline([
    ('preprocessor', preprocessor),
    ('regressor', stacked)
])

# Train/test split
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=22
)
#randome_state=22,1000, 93
model_pipeline.fit(X_train, y_train)

# Evaluate
y_pred = model_pipeline.predict(X_test)
r2 = r2_score(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)

print(f"Model Accuracy (R2): {r2:.3f}")
print(f"Error Margin (MAE): {mae:.3f} tons/ha")

joblib.dump(model_pipeline, "../models/maize_model.pkl")
print("Model saved.")


Model Accuracy (R2): 0.918
Error Margin (MAE): 0.076 tons/ha
Model saved.


In [18]:
import os
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# -------------------------
# Setup
# -------------------------
REPORT_DIR = "../reports"
os.makedirs(REPORT_DIR, exist_ok=True)

sns.set_theme(style="darkgrid")


# -------------------------
# Plot functions
# -------------------------
def plot_national_yield_trend(df, save_dir=REPORT_DIR):
    yr_df = (
        df.groupby("Year", as_index=False)["Yield"]
          .mean()
    )

    plt.figure(figsize=(8, 4))
    sns.lineplot(data=yr_df, x="Year", y="Yield", marker="o")
    plt.title("National Average Maize Yield by Year")
    plt.xlabel("Year")
    plt.ylabel("Yield (tons/ha)")
    plt.savefig(f"{save_dir}/national_yield_trend.png", bbox_inches="tight")
    plt.close()


def plot_yield_by_soil_type(df, save_dir=REPORT_DIR):
    plt.figure(figsize=(10, 6))
    sns.boxplot(data=df, x="Soil_Type", y="Yield")
    plt.xticks(rotation=45, ha="right")
    plt.title("Yield Distribution by Soil Type")
    plt.xlabel("Soil Type")
    plt.ylabel("Yield (tons/ha)")
    plt.savefig(f"{save_dir}/yield_by_soil_type.png", bbox_inches="tight")
    plt.close()


def plot_rainfall_vs_yield(df, save_dir=REPORT_DIR):
    plt.figure(figsize=(8, 6))
    sns.regplot(
        data=df,
        x="Rainfall",
        y="Yield",
        scatter_kws={"s": 10},
        line_kws={"color": "red"}
    )
    plt.title("Rainfall (Apr–Aug) vs Yield")
    plt.xlabel("Rainfall (mm)")
    plt.ylabel("Yield (tons/ha)")
    plt.savefig(f"{save_dir}/rainfall_vs_yield.png", bbox_inches="tight")
    plt.close()


def plot_feature_correlation(df, save_dir=REPORT_DIR):
    num_cols = [
        "Yield",
        "Rainfall",
        "Temperature",
        "Humidity",
        "Sunlight",
        "Soil_Moisture",
        "Yield_Lag1"
    ]

    corr = df[num_cols].corr()

    plt.figure(figsize=(8, 6))
    sns.heatmap(corr, annot=True, fmt=".2f", cmap="coolwarm")
    plt.title("Feature Correlation Matrix")
    plt.savefig(f"{save_dir}/feature_correlation.png", bbox_inches="tight")
    plt.close()


# -------------------------
# Optional: Faceted plot
# -------------------------
def plot_yield_trend_by_soil(df, save_dir=REPORT_DIR):
    g = sns.relplot(
        data=df,
        x="Year",
        y="Yield",
        col="Soil_Type",
        kind="line",
        col_wrap=3,
        height=3,
        facet_kws={"sharey": True}
    )
    g.set_titles("{col_name}")
    g.set_axis_labels("Year", "Yield (tons/ha)")
    g.fig.suptitle("Yield Trends by Soil Type", y=1.05)
    g.savefig(f"{save_dir}/yield_trend_by_soil_type.png", bbox_inches="tight")
    plt.close()


# -------------------------
# Main runner
# -------------------------
def run_all_plots(df):
    plot_national_yield_trend(df)
    plot_yield_by_soil_type(df)
    plot_rainfall_vs_yield(df)
    plot_feature_correlation(df)
    plot_yield_trend_by_soil(df)


if __name__ == "__main__":
    df = pd.read_csv("../data/final/Ghana_Maize.csv")
    run_all_plots(df)
    print("All visualizations saved to ../reports/")


All visualizations saved to ../reports/
